In [2]:
import numpy as np
import pandas as pd
import sklearn
import plotly
import plotly.express as px

In [3]:
df = pd.read_csv("/home/srimanth/Programs/project_4-2/ytube_clean.csv")
df.head() 

,Unnamed: 0,sl_no,video_id,user_id,comment,likes,last_update_time,sentiment,sentiment_label,stemmed_text,stemmed_clean_text
0,0,1,IBe2o-cZncU,UCETH_ePd6_jXtrT9hvJ8EAg,Singularity: The day you wanna turn off your p...,58,2019-12-14T03:30:13.000Z,0.0000,neutral,singular the day you wan na turn off your pc b...,"['singular', 'day', 'wan', 'turn', 'pc', 'refus']"
1,1,2,IBe2o-cZncU,UCVYV59yMBaZfUfbKRvIrsGQ,Jack Ma AI: Alibaba Intelligence,930,2019-12-09T13:00:25.000Z,0.0000,neutral,jack Ma AI alibaba intellig,"['jack', 'Ma', 'AI', 'alibaba', 'intellig']"
2,2,3,IBe2o-cZncU,UCw40cwbkNDrFpdd4YUig3iw,Alan Turing deserve recognition for his work o...,136,2019-12-09T13:34:53.000Z,0.4000,positive,alan ture deserv recognit for hi work on learn...,"['alan', 'ture', 'deserv', 'recognit', 'hi', '..."
3,3,4,IBe2o-cZncU,UCmrcSjCCikF4I68gMdRV64A,Your production house gives us the fascinating...,653,2019-12-09T12:54:02.000Z,0.5125,positive,your product hous give us the fascin tech vide...,"['product', 'hous', 'give', 'us', 'fascin', 't..."
4,4,5,IBe2o-cZncU,UChC_nPFZN2_OwogxYTC_TMQ,Sarah Connor is watching this and making a list,126,2019-12-09T15:16:38.000Z,0.0000,neutral,sarah connor is watch thi and make a list,"['sarah', 'connor', 'watch', 'make', 'list']"


In [4]:
from sklearn import feature_extraction

corpus = list(df['stemmed_text'])
corpus = [item.lower() for item in corpus]

vectorizer = feature_extraction.text.CountVectorizer(analyzer='word', stop_words='english')
bow = vectorizer.fit_transform(corpus)
feature_list = vectorizer.get_feature_names()

X_bow = pd.DataFrame(bow.toarray(),columns=vectorizer.get_feature_names())
y = df["sentiment_label"] 

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=0) 

In [6]:
X_train.shape 

(1193, 3499)

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
bool_array = (model.predict(X_test) == np.array(y_test))
print(len(bool_array))
count = np.count_nonzero(bool_array)
print("True: {}, False: {}".format(count, len(bool_array)-count)) 

299
True: 227, False: 72


In [9]:
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7)  

tf_idf = tfidf_vectorizer.fit_transform(corpus)
feature_list = tfidf_vectorizer.get_feature_names()
X_tf_idf = pd.DataFrame(tf_idf.toarray(),columns=feature_list)  

X_tf_idf.shape

(1492, 843)

In [14]:
from sklearn.model_selection import train_test_split
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_bow, y, test_size=0.2, random_state=0) 

In [15]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_tf, y_train_tf)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
bool_array = (model.predict(X_test_tf) == np.array(y_test_tf))
print(len(bool_array))
count = np.count_nonzero(bool_array)
print("True: {}, False: {}".format(count, len(bool_array)-count)) 

299
True: 227, False: 72


In [17]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def metrics(predictions, y_test):
    print(confusion_matrix(y_test,predictions))  
    print(classification_report(y_test,predictions))  
    print(accuracy_score(y_test, predictions))

In [18]:
metrics(y_test, model.predict(X_test))

[[  0   1   0]
 [  6 183  37]
 [  0  28  44]]
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
     neutral       0.86      0.81      0.84       226
    positive       0.54      0.61      0.58        72

    accuracy                           0.76       299
   macro avg       0.47      0.47      0.47       299
weighted avg       0.78      0.76      0.77       299

0.7591973244147158


In [19]:
metrics(y_test_tf, model.predict(X_test_tf))

[[  0   1   0]
 [  6 183  37]
 [  0  28  44]]
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
     neutral       0.86      0.81      0.84       226
    positive       0.54      0.61      0.58        72

    accuracy                           0.76       299
   macro avg       0.47      0.47      0.47       299
weighted avg       0.78      0.76      0.77       299

0.7591973244147158
